### The S3 Ingestionator
Ingest Cloudtrail logs, or something else, directly from S3 into a dataframe. Part of the DUNE project (https://github.com/opendr-io/dune) for hunting threats that are resistant to conventional detection. Run this in the same region!  Ppulate keys if necessary and the target S3 URI below. 

In [ ]:
import io, gzip, json, pandas as pd
import re, io, json, gzip
import boto3
import pandas as pd
import re, io, json, gzip, boto3


In [2]:
AWS_ACCESS_KEY_ID = "key"
AWS_SECRET_ACCESS_KEY = "secret"

S3_URI = "s3://aws-cloudtrail-logs-etc"


In [9]:

m = re.match(r"^s3://([^/]+)/(.*)$", S3_URI.rstrip("/"))
S3_BUCKET, S3_PREFIX = (m.group(1), m.group(2) + "/") if m else (S3_URI[5:], "")

s3 = boto3.client("s3")
paginator = s3.get_paginator("list_objects_v2")

all_objs = []
for page in paginator.paginate(Bucket=S3_BUCKET, Prefix=S3_PREFIX):
    for obj in page.get("Contents", []):
        if obj["Key"].endswith(".json.gz"):
            all_objs.append(obj)

print(f"Found {len(all_objs)} files under {S3_URI}")

total_bytes = sum(obj["Size"] for obj in all_objs)
total_mb = total_bytes / (1024 ** 2)
total_gb = total_bytes / (1024 ** 3)

print(f"Total size: {total_bytes:,} bytes")
print(f"≈ {total_mb:.2f} MB  ({total_gb:.3f} GB)")


Found 261 files under s3://aws-cloudtrail-logs-637602092272-128dfcf0/AWSLogs/637602092272/CloudTrail/us-east-1/2025/10/
Total size: 1,196,327 bytes
≈ 1.14 MB  (0.001 GB)


In [10]:

def list_cloudtrail_fields(s3_uri, access_key=None, secret_key=None, limit=10):
    m = re.match(r"^s3://([^/]+)/?(.*)$", s3_uri.rstrip("/"))
    if not m:
        raise ValueError(f"Invalid S3 URI: {s3_uri}")
    bucket, prefix = m.group(1), (m.group(2) + "/") if m.group(2) else ""

    s3 = boto3.client(
        "s3",
        aws_access_key_id=access_key,
        aws_secret_access_key=secret_key,
    )

    paginator = s3.get_paginator("list_objects_v2")
    field_names = set()
    files_read = 0

    for page in paginator.paginate(Bucket=bucket, Prefix=prefix):
        for obj in page.get("Contents", []):
            if not obj["Key"].endswith(".json.gz"):
                continue

            body = s3.get_object(Bucket=bucket, Key=obj["Key"])["Body"].read()
            with gzip.GzipFile(fileobj=io.BytesIO(body)) as gz:
                payload = json.loads(gz.read().decode("utf-8"))
                for rec in payload.get("Records", []):
                    field_names.update(rec.keys())

            files_read += 1
            if limit and files_read >= limit:
                break
        if limit and files_read >= limit:
            break

    return sorted(field_names)

# --- Example ---
fields = list_cloudtrail_fields(
    S3_URI,
    access_key=AWS_ACCESS_KEY_ID,
    secret_key=AWS_SECRET_ACCESS_KEY,
    limit=5,   # read first 5 files to sample
)
print(f"Discovered {len(fields)} top-level fields:")
for f in fields:
    print("-", f)


Discovered 27 top-level fields:
- additionalEventData
- apiVersion
- awsRegion
- errorCode
- errorMessage
- eventCategory
- eventID
- eventName
- eventSource
- eventTime
- eventType
- eventVersion
- managementEvent
- readOnly
- recipientAccountId
- requestID
- requestParameters
- resources
- responseElements
- sessionCredentialFromConsole
- sharedEventID
- sourceIPAddress
- tlsDetails
- userAgent
- userIdentity
- vpcEndpointAccountId
- vpcEndpointId


In [11]:

def _parse_s3_uri(s3_uri: str):
    m = re.match(r"^s3://([^/]+)/?(.*)$", s3_uri.rstrip("/"))
    if not m:
        raise ValueError(f"Invalid S3 URI: {s3_uri}")
    bucket, prefix = m.group(1), (m.group(2) + "/") if m.group(2) else ""
    return bucket, prefix

def _make_s3_client(access_key=None, secret_key=None, session_token=None, region_name=None):
    # Falls back to default credential chain if keys are None
    session = boto3.Session(
        aws_access_key_id=access_key,
        aws_secret_access_key=secret_key,
        aws_session_token=session_token,
        region_name=region_name,
    )
    return session.client("s3")

def load_cloudtrail_df_from_s3_uri(
    s3_uri: str,
    access_key=None,
    secret_key=None,
    session_token=None,
    region_name=None,
    suffix=".json.gz",
    limit=None,             # set an int to sample files first
    select_cols=None,       # list of columns to keep after normalize
    datetime_col="eventTime"
) -> pd.DataFrame:
    """
    Recursively loads CloudTrail gzipped JSON logs from an S3 URI into a flattened DataFrame.
    """
    bucket, prefix = _parse_s3_uri(s3_uri)
    s3 = _make_s3_client(access_key, secret_key, session_token, region_name)

    paginator = s3.get_paginator("list_objects_v2")
    records = []
    file_count = 0

    for page in paginator.paginate(Bucket=bucket, Prefix=prefix):
        contents = page.get("Contents", [])
        for obj in contents:
            key = obj["Key"]
            if not key.endswith(suffix):
                continue

            file_count += 1
            if limit and file_count > limit:
                break

            body = s3.get_object(Bucket=bucket, Key=key)["Body"].read()
            with gzip.GzipFile(fileobj=io.BytesIO(body)) as gz:
                try:
                    payload = json.loads(gz.read().decode("utf-8"))
                except json.JSONDecodeError:
                    # Skip malformed objects but keep going
                    continue

            recs = payload.get("Records", [])
            if recs:
                records.extend(recs)
        if limit and file_count >= limit:
            break

    if not records:
        return pd.DataFrame()

    df = pd.json_normalize(records, sep=".")

    # Optional column pruning
    if select_cols:
        keep = [c for c in select_cols if c in df.columns]
        if keep:
            df = df[keep]

    # Parse eventTime if present
    if datetime_col in df.columns:
        df[datetime_col] = pd.to_datetime(df[datetime_col], utc=True, errors="coerce")

    return df


In [12]:

df = load_cloudtrail_df_from_s3_uri(
    S3_URI,
    access_key=AWS_ACCESS_KEY_ID,
    secret_key=AWS_SECRET_ACCESS_KEY,
    region_name="us-east-1",  # optional
    select_cols=[
        "eventTime",
        "eventName",
        "eventSource",
        "awsRegion",
        "sourceIPAddress",
        "userIdentity.type",
        "userIdentity.accountId",
        "userIdentity.arn",
        "userAgent",
        "requestParameters",
        "responseElements",
        "errorCode",
        "errorMessage",
        "resources"
    ],
)

print(df.shape)
df.head()


(3477, 14)


,eventTime,eventName,eventSource,awsRegion,sourceIPAddress,userIdentity.type,userIdentity.accountId,userIdentity.arn,userAgent,requestParameters,responseElements,errorCode,errorMessage,resources
0,2025-10-12 02:13:08+00:00,DescribeRegions,ec2.amazonaws.com,us-east-1,71.184.228.151,Root,637602092272,arn:aws:iam::637602092272:root,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:1...,NaN,NaN,NaN,NaN,NaN
1,2025-10-12 02:13:08+00:00,GetFeatureOptInStatus,notifications.amazonaws.com,us-east-1,71.184.228.151,Root,637602092272,arn:aws:iam::637602092272:root,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:1...,NaN,NaN,NaN,NaN,NaN
2,2025-10-12 02:13:02+00:00,ConsoleLogin,signin.amazonaws.com,us-east-1,71.184.228.151,Root,637602092272,arn:aws:iam::637602092272:root,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:1...,NaN,NaN,NaN,NaN,NaN
3,2025-10-12 02:13:11+00:00,DescribeSnapshots,ec2.amazonaws.com,us-east-1,71.184.228.151,Root,637602092272,arn:aws:iam::637602092272:root,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:1...,NaN,NaN,NaN,NaN,NaN
4,2025-10-12 02:13:11+00:00,DescribeVolumes,ec2.amazonaws.com,us-east-1,71.184.228.151,Root,637602092272,arn:aws:iam::637602092272:root,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:1...,NaN,NaN,NaN,NaN,NaN


In [13]:

def summarize_sparse_columns(df, threshold=0.95):
    total_rows = len(df)
    nan_counts = df.isna().sum()

    # Entirely NaN
    all_nan = nan_counts[nan_counts == total_rows].index.tolist()

    # Mostly NaN (but not entirely)
    mostly_nan = nan_counts[(nan_counts / total_rows >= threshold) & (nan_counts < total_rows)].index.tolist()

    print(f"Total rows: {total_rows:,}")
    print(f"\n🧱 Columns entirely NaN ({len(all_nan)}):")
    for c in all_nan:
        print("  -", c)

    print(f"\n🌫️ Columns ≥ {int(threshold*100)}% NaN ({len(mostly_nan)}):")
    for c in mostly_nan:
        pct = nan_counts[c] / total_rows * 100
        print(f"  - {c}: {pct:.1f}% NaN")

    return all_nan, mostly_nan

# --- Example usage ---
all_nan_cols, mostly_nan_cols = summarize_sparse_columns(df, threshold=0.95)

empty_cols = [col for col in df.columns if df[col].isna().all()]
print()
print(f"{len(empty_cols)} columns are entirely NaN:")
for c in empty_cols:
    print("-", c)
    
print()
df.info(verbose=True)


Total rows: 3,477

🧱 Columns entirely NaN (2):
  - requestParameters
  - responseElements

🌫️ Columns ≥ 95% NaN (2):
  - errorCode: 98.0% NaN
  - errorMessage: 98.3% NaN

2 columns are entirely NaN:
- requestParameters
- responseElements

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3477 entries, 0 to 3476
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype              
---  ------                  --------------  -----              
 0   eventTime               3477 non-null   datetime64[ns, UTC]
 1   eventName               3477 non-null   object             
 2   eventSource             3477 non-null   object             
 3   awsRegion               3477 non-null   object             
 4   sourceIPAddress         3477 non-null   object             
 5   userIdentity.type       3473 non-null   object             
 6   userIdentity.accountId  2496 non-null   object             
 7   userIdentity.arn        2492 non-null   object             
 8  